Etape 1: Import et paramètres

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import tensorflow.keras.datasets.imdb as imdb
import pandas as pd
from sklearn.metrics import confusion_matrix

import os,sys
from importlib import reload

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("CPU is", "available" if tf.config.list_physical_devices('CPU') else "NOT AVAILABLE")

1.1 Fixation des paramètres

In [ ]:
vocab_size           = 10000
hide_most_frequently = 0

epochs     = 10
batch_size = 512


1.2Comprendre le 'One-hot encoding'

In [ ]:
sentence = "I've never seen a movie like this before"

dictionary  = {"a":0, "before":1, "fantastic":2, "i've":3, "is":4, "like":5, "movie":6, "never":7, "seen":8, "this":9}

In [ ]:
sentence_words = sentence.lower().split()

sentence_vect  = [ dictionary[w] for w in sentence_words ]

print('Words sentence are         : ', sentence_words)
print('Our vectorized sentence is : ', sentence_vect)

Ensuite, nous codons notre phrase vectorisée comme un tenseur :

In [ ]:
# ---- We get a (sentence length x vector size) matrix of zeros
#
onehot = np.zeros( (10,8) )

# ---- We set some 1 for each word
#
for i,w in enumerate(sentence_vect):
    onehot[w,i]=1

# --- Show it
#
print('In a basic way :\n\n', onehot, '\n\nWith a pandas wiew :\n')
data={ f'{sentence_words[i]:.^10}':onehot[:,i] for i,w in enumerate(sentence_vect) }
df=pd.DataFrame(data)
df.index=dictionary.keys()
df.style.set_precision(0).highlight_max(axis=0).set_properties(**{'text-align': 'center'})

Etape 2.
1. Récuperation de données

L'ensemble de données est composé de 2 parties :

•	Pour les critiques, ce sera notre x ;

•	Pour les avis (positifs/négatifs), ce sera notre y.

Il y a aussi un dictionnaire, car les mots sont indexés dans les commentaires.


In [ ]:

train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [ ]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))
print("\n")
train_examples[:10]

Etape 2
2. Load Dataset

In [ ]:
# ----- Retrieve x,y
#
(x_train, y_train), (x_test, y_test) = imdb.load_data( num_words=vocab_size, skip_top=hide_most_frequently)

y_train = np.asarray(y_train).astype('float32')
y_test  = np.asarray(y_test ).astype('float32')

# ---- About

print("x_train : {}  y_train : {}".format(x_train.shape, y_train.shape))
print("x_test  : {}  y_test  : {}".format(x_test.shape,  y_test.shape))

Etape 3: A propos de notre Dataset

3.1.Codages des phrases

In [ ]:
print('\nReview example (x_train[12]) :\n\n',x_train[12])
print('\nOpinions (y_train) :\n\n',y_train)

3.2.Chargement du dictionnaire 

In [ ]:
# ---- Retrieve dictionary {word:index}, and encode it in ascii
#
word_index = imdb.get_word_index()

# ---- Shift the dictionary from +3
#
word_index = {w:(i+3) for w,i in word_index.items()}

# ---- Add <pad>, <start> and <unknown> tags
#
word_index.update( {'<pad>':0, '<start>':1, '<unknown>':2, '<undef>':3,} )

# ---- Create a reverse dictionary : {index:word}
#
index_word = {index:word for word,index in word_index.items()} 

# ---- About dictionary
#
print('\nDictionary size     : ', len(word_index))
print('\nSmall extract :\n')
for k in range(440,460):print(f'    {k:2d} : {index_word[k]}' )

# ---- Add a nice function to transpose :
#
def dataset2text(review):
    return ' '.join([index_word.get(i, '?') for i in review])

3.3 Jetez un coup d'œil, pour les humains

In [ ]:
print('Review example :')
print(x_train[12])
print("\n")
print('After translation:')
print(dataset2text(x_train[12]))


3.4. Quelques statistiques

In [ ]:
sizes=[len(i) for i in x_train]
plt.figure(figsize=(16,6))
plt.hist(sizes, bins=400)
plt.gca().set(title='Distribution of reviews by size - [{:5.2f}, {:5.2f}]'.format(min(sizes),max(sizes)), 
              xlabel='Size', ylabel='Density', xlim=[0,1500])
#pwk.save_fig('01-stats-sizes')
plt.show()

In [ ]:
unk=[ 100*(s.count(2)/len(s)) for s in x_train]
plt.figure(figsize=(16,6))
plt.hist(unk, bins=100)
plt.gca().set(title='Percent of unknown words - [{:5.2f}, {:5.2f}]'.format(min(unk),max(unk)), 
              xlabel='# unknown', ylabel='Density', xlim=[0,30])
#pwk.save_fig('02-stats-unknown')
plt.show()

Etape 4:
Approche de base.

Chaque phrase est codée avec un vecteur de longueur égale à la taille du dictionnaire.

Ainsi, chaque phrase sera codée avec un vecteur simple.
La valeur de chaque composant est 0 si le mot n'est pas présent dans la phrase ou 1 si le mot est présent.

Pour une phrase s=[3,4,7] et un dictionnaire de 10 mots...
Nous aurons un vecteur v=[0,0,0,1,1,0,0,1,0,0,0,0]

4.1 Our one hot encoder

In [ ]:
def one_hot_encoder(x, vector_size=10000):
    
    # ---- Set all to 0
    #
    x_encoded = np.zeros((len(x), vector_size))
    
    # ---- For each sentence
    #
    for i,sentence in enumerate(x):
        for word in sentence:
            x_encoded[i, word] = 1.

    return x_encoded

4.2. Encoding

In [ ]:
x_train = one_hot_encoder(x_train)
x_test  = one_hot_encoder(x_test)

print("To have a look, x_train[12] became :", x_train[12] )

Etape 5:Build the model

In [ ]:
def get_model(vector_size=10000):
    
    model = keras.Sequential()
    model.add(keras.layers.Input( shape=(vector_size,) ))
    model.add(keras.layers.Dense( 32, activation='relu'))
    model.add(keras.layers.Dense( 32, activation='relu'))
    model.add(keras.layers.Dense( 1, activation='sigmoid'))
    
    model.compile(optimizer = 'rmsprop',
                  loss      = 'binary_crossentropy',
                  metrics   = ['accuracy'])
    return model

6.Train the model

6.1 Get it

In [ ]:
model = get_model(vector_size=vocab_size)

model.summary()

6.2 Add Callback

In [ ]:
os.makedirs(f'/models',   mode=0o750, exist_ok=True)
save_dir = f'/models/best_model.h5'
savemodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0, save_best_only=True)

6.3 Train it

In [ ]:
%%time

history = model.fit(x_train,
                    y_train,
                    epochs          = epochs,
                    batch_size      = batch_size,
                    validation_data = (x_test, y_test),
                    verbose         = 1,
                    callbacks       = [savemodel_callback])


Etape 7: Evaluation du résultat

7.1Historique de l'entrainement et création d'un graphique d'évaluation

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Dans ce graphique, les points représentent la perte et la précision de la formation, et les lignes pleines représentent la perte de validation et la précision.

Etape 8: Reload and evaluate best model

In [ ]:
model = keras.models.load_model(f'/models/best_model.h5')

# ---- Evaluate
score  = model.evaluate(x_test, y_test, verbose=0)

print('x_test / loss      : {:5.4f}'.format(score[0]))
print('x_test / accuracy  : {:5.4f}'.format(score[1]))

values=[score[1], 1-score[1]]


# ---- Confusion matrix

y_sigmoid = model.predict(x_test)

y_pred = y_sigmoid.copy()
y_pred[ y_sigmoid< 0.5 ] = 0
y_pred[ y_sigmoid>=0.5 ] = 1    

confusion_matrix(y_test,y_pred,labels=range(2))
confusion_matrix(y_test,y_pred,range(2), normalize='true' )